In [ ]:

import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import drive

# ==========================================
# 1. GOOGLE DRIVE BAĞLANTISI
# ==========================================
# Eğer drive zaten bağlı değilse bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- DİKKAT: Buranın images/ kısmının sonuna hata türünü yazınız: (saglam,bozuk,delik,kalinti,yarik)
FOLDER_PATH = "/content/drive/MyDrive/images/saglam"
# ==========================================

# ==========================================
# 2. SABİT PARAMETRELER (AYARLAR)
# ==========================================
# --- Genel Ayarlar ---
MAX_SEARCH_WIDTH = 127      # Kaynak dikişinin aranacağı maksimum genişlik
FIXED_TIP_THRESH = 200      # Kaynak ucunu bulmak için parlaklık eşiği
OFFSET_VAL = 10             # Kaynak ucundan sonra başlama boşluğu

# --- Görüntü İşleme / Hata Bulma ---
COMMON_SENSITIVITY = 5      # Adaptive Threshold (Düşük = Daha hassas)
COMMON_CONTRAST = 2.0       # CLAHE Clip Limit
MORPH_SIZE = 1              # Gürültü temizleme boyutu
MIN_DEFECT_AREA = 127       # Min hata alanı

# --- Kenar Duvarı Mantığı ---
CENTER_SAFE_ZONE = 40       # Merkeze ±40px mesafedeki her şey HATA kabul edilir.
EDGE_MIN_HEIGHT = 60        # Dışarıdaki şeklin "Duvar" sayılması için min boy.

# --- Parlaklık / Glare ---
BRIGHT_THRESH = 130         # Parlak yerler maskelenir
IGNORE_RADIUS_BRIGHT = 10   # Parlak alan güvenlik çemberi

# --- Sınıflandırma ---
CIRCULARITY_THRESH = 0.67   # Yuvarlaklık eşiği
CRACK_RATIO = 3.9           # En/Boy oranı

# ==========================================
# 3. İŞLEM FONKSİYONU
# ==========================================
def process_images_in_colab():
    if not os.path.exists(FOLDER_PATH):
        print(f"HATA: Klasör bulunamadı -> {FOLDER_PATH}")
        print("Lütfen FOLDER_PATH değişkenini doğru ayarlayın.")
        return

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    image_files = [f for f in os.listdir(FOLDER_PATH) if f.lower().endswith(valid_extensions)]
    image_files.sort()

    if not image_files:
        print("Klasörde resim bulunamadı.")
        return

    print(f"Toplam {len(image_files)} resim bulundu. Analiz başlıyor...\n")

    for img_name in image_files:
        full_path = os.path.join(FOLDER_PATH, img_name)

        current_image = cv2.imread(full_path)

        if current_image is None:
            print(f"Resim okunamadı: {img_name}")
            continue

        if current_image.shape[0] > 800:
            scale = 800 / current_image.shape[0]
            dim = (int(current_image.shape[1] * scale), 800)
            current_image = cv2.resize(current_image, dim)

        display_img = current_image.copy()
        h, w = current_image.shape[:2]
        gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

        # ------------------------------------------------------------
        # 1. ADIM: KAYNAK UCUNU BUL
        # ------------------------------------------------------------
        _, mask_tip = cv2.threshold(gray, FIXED_TIP_THRESH, 255, cv2.THRESH_BINARY)
        contours_tip, _ = cv2.findContours(mask_tip, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        found_tip = False
        scan_start_y = 0
        tip_center_x = 0

        limit_outer_left = 0
        limit_outer_right = w
        final_left = 0
        final_right = w

        if contours_tip:
            largest_tip = max(contours_tip, key=cv2.contourArea)
            if cv2.contourArea(largest_tip) > 20:
                tx, ty, tw, th = cv2.boundingRect(largest_tip)
                tip_center_x = tx + tw // 2
                found_tip = True

                limit_outer_left = max(0, tip_center_x - MAX_SEARCH_WIDTH)
                limit_outer_right = min(w, tip_center_x + MAX_SEARCH_WIDTH)

                final_left = limit_outer_left
                final_right = limit_outer_right

                # Dinamik Başlangıç
                search_y_start = ty + th + OFFSET_VAL

                if search_y_start < h:
                    roi_glare_search = gray[search_y_start:h, int(limit_outer_left):int(limit_outer_right)]
                    if roi_glare_search.size > 0:
                        mask_glare = roi_glare_search > BRIGHT_THRESH
                        if np.any(mask_glare):
                            k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                            k_dilate = np.ones((k_dim, k_dim), np.uint8)
                            mask_glare_dilated = cv2.dilate(mask_glare.astype(np.uint8), k_dilate, iterations=1)
                            glare_ys, _ = np.where(mask_glare_dilated > 0)
                            if glare_ys.size > 0:
                                max_glare_y_local = np.max(glare_ys)
                                scan_start_y = search_y_start + max_glare_y_local
                            else:
                                scan_start_y = search_y_start
                        else:
                            scan_start_y = search_y_start
                    else:
                        scan_start_y = search_y_start
                else:
                    scan_start_y = h

                scan_start_y = min(scan_start_y, h - 1)
                scan_start_y = int(scan_start_y)
                cv2.rectangle(display_img, (tx, ty), (tx+tw, ty+th), (0, 0, 255), 2)

        # ------------------------------------------------------------
        # 2. ADIM: ANALİZ VE SINIR BELİRLEME
        # ------------------------------------------------------------
        debug_mask = np.zeros_like(gray)
        defect_count = 0

        if found_tip and scan_start_y < h:
            roi_gray = gray[scan_start_y:h, int(limit_outer_left):int(limit_outer_right)]

            if roi_gray.size > 0:
                h_roi, w_roi = roi_gray.shape

                # Görüntü İşleme
                clahe = cv2.createCLAHE(clipLimit=COMMON_CONTRAST, tileGridSize=(8,8))
                roi_enhanced = clahe.apply(roi_gray)
                roi_blurred = cv2.medianBlur(roi_enhanced, 5)
                roi_binary = cv2.adaptiveThreshold(
                    roi_blurred, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV, 21, COMMON_SENSITIVITY
                )

                if MORPH_SIZE > 0:
                    k_size = 2 * MORPH_SIZE + 1
                    kernel_morph = np.ones((k_size, k_size), np.uint8)
                    roi_binary = cv2.morphologyEx(roi_binary, cv2.MORPH_OPEN, kernel_morph)

                # Parlaklık Maskesi
                mask_bright_raw = roi_gray > BRIGHT_THRESH
                mask_exclude_bright = np.zeros_like(roi_gray, dtype=np.uint8)
                if np.any(mask_bright_raw):
                    k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                    kernel_dilate = np.ones((k_dim, k_dim), np.uint8)
                    mask_bright_uint8 = (mask_bright_raw * 255).astype(np.uint8)
                    mask_exclude_bright = cv2.dilate(mask_bright_uint8, kernel_dilate, iterations=1)

                # Sınırları Belirle
                roi_center_x = w_roi // 2
                safe_zone_start = roi_center_x - CENTER_SAFE_ZONE
                safe_zone_end = roi_center_x + CENTER_SAFE_ZONE

                # Gri Referans Çizgiler
                cv2.line(display_img, (int(limit_outer_left + safe_zone_start), scan_start_y),
                         (int(limit_outer_left + safe_zone_start), h), (80,80,80), 1)
                cv2.line(display_img, (int(limit_outer_left + safe_zone_end), scan_start_y),
                         (int(limit_outer_left + safe_zone_end), h), (80,80,80), 1)

                cnts, _ = cv2.findContours(roi_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                left_wall_candidates = []
                right_wall_candidates = []
                defects_to_process = []

                for cnt in cnts:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    cnt_center_x = bx + bw // 2

                    if mask_exclude_bright[by + bh//2, cnt_center_x] > 0: continue
                    if cv2.contourArea(cnt) < MIN_DEFECT_AREA: continue

                    # Sınıflandırma
                    if (bx + bw) < safe_zone_start:
                        if bh > EDGE_MIN_HEIGHT:
                            left_wall_candidates.append(bx + bw)
                    elif bx > safe_zone_end:
                        if bh > EDGE_MIN_HEIGHT:
                            right_wall_candidates.append(bx)
                    else:
                        defects_to_process.append(cnt)

                # Final Sınırlar
                if left_wall_candidates:
                    final_left = limit_outer_left + max(left_wall_candidates)
                else:
                    final_left = limit_outer_left

                if right_wall_candidates:
                    final_right = limit_outer_left + min(right_wall_candidates)
                else:
                    final_right = limit_outer_right

                # Hataları Çiz
                for cnt in defects_to_process:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    global_bx = limit_outer_left + bx
                    global_cnt_center = global_bx + bw // 2

                    if final_left < global_cnt_center < final_right:
                        defect_count += 1

                        cnt_shifted = cnt + np.array([int(limit_outer_left), scan_start_y])

                        perimeter = cv2.arcLength(cnt, True)
                        if perimeter == 0: continue
                        area = cv2.contourArea(cnt)
                        circularity = 4 * np.pi * (area / (perimeter * perimeter))
                        aspect_ratio = float(bw) / bh if bh > 0 else 0

                        label = ""
                        color = (0,0,0)
                        if circularity > CIRCULARITY_THRESH:
                            label = "Gozenek"
                            color = (0, 0, 255)
                            (cx, cy), radius = cv2.minEnclosingCircle(cnt_shifted)
                            cv2.circle(display_img, (int(cx), int(cy)), int(radius)+2, color, 2)
                        elif aspect_ratio > CRACK_RATIO or aspect_ratio < (1/CRACK_RATIO):
                            label = "Catlak"
                            color = (0, 255, 255)
                            cv2.rectangle(display_img, (global_bx, by+scan_start_y), (global_bx+bw, by+bh+scan_start_y), color, 2)
                        else:
                            label = "Bozuk"
                            color = (255, 0, 255)
                            cv2.drawContours(display_img, [cnt_shifted], -1, color, 2)

                        cv2.putText(display_img, label, (global_bx, by+scan_start_y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # ------------------------------------------------------------
        # 3. GÖRSELLEŞTİRME
        # ------------------------------------------------------------
        if scan_start_y > 0:
            overlay_top = display_img.copy()
            cv2.rectangle(overlay_top, (0, 0), (w, scan_start_y), (40, 40, 40), -1)
            cv2.addWeighted(overlay_top, 0.3, display_img, 0.7, 0, display_img)
            cv2.line(display_img, (0, scan_start_y), (w, scan_start_y), (0, 0, 255), 2)

        cv2.line(display_img, (int(final_left), scan_start_y), (int(final_left), h), (0, 165, 255), 3)
        cv2.line(display_img, (int(final_right), scan_start_y), (int(final_right), h), (0, 165, 255), 3)

        debug_view = cv2.cvtColor(debug_mask, cv2.COLOR_GRAY2BGR)

        # Bilgi Yazısı
        cv2.rectangle(display_img, (0, 0), (w, 40), (0,0,0), -1)
        info_text = f"Dosya: {img_name} | Hata Sayisi: {defect_count}"
        cv2.putText(display_img, info_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        combined = np.hstack((debug_view, display_img))

        # Ekrana Bas (Colab)
        print(f"\n--- {img_name} Analizi ---")
        cv2_imshow(combined)

if __name__ == "__main__":
    process_images_in_colab()

Output hidden; open in https://colab.research.google.com to view.

In [1]:

import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import drive

# ==========================================
# 1. GOOGLE DRIVE BAĞLANTISI
# ==========================================
# Eğer drive zaten bağlı değilse bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- DİKKAT: Buranın images/ kısmının sonuna hata türünü yazınız: (saglam,bozuk,delik,kalinti,yarik)
FOLDER_PATH = "/content/drive/MyDrive/images/delik"
# ==========================================

# ==========================================
# 2. SABİT PARAMETRELER (AYARLAR)
# ==========================================
# --- Genel Ayarlar ---
MAX_SEARCH_WIDTH = 127      # Kaynak dikişinin aranacağı maksimum genişlik
FIXED_TIP_THRESH = 200      # Kaynak ucunu bulmak için parlaklık eşiği
OFFSET_VAL = 10             # Kaynak ucundan sonra başlama boşluğu

# --- Görüntü İşleme / Hata Bulma ---
COMMON_SENSITIVITY = 5      # Adaptive Threshold (Düşük = Daha hassas)
COMMON_CONTRAST = 2.0       # CLAHE Clip Limit
MORPH_SIZE = 1              # Gürültü temizleme boyutu
MIN_DEFECT_AREA = 127       # Min hata alanı

# --- Kenar Duvarı Mantığı ---
CENTER_SAFE_ZONE = 40       # Merkeze ±40px mesafedeki her şey HATA kabul edilir.
EDGE_MIN_HEIGHT = 60        # Dışarıdaki şeklin "Duvar" sayılması için min boy.

# --- Parlaklık / Glare ---
BRIGHT_THRESH = 130         # Parlak yerler maskelenir
IGNORE_RADIUS_BRIGHT = 10   # Parlak alan güvenlik çemberi

# --- Sınıflandırma ---
CIRCULARITY_THRESH = 0.67   # Yuvarlaklık eşiği
CRACK_RATIO = 3.9           # En/Boy oranı

# ==========================================
# 3. İŞLEM FONKSİYONU
# ==========================================
def process_images_in_colab():
    if not os.path.exists(FOLDER_PATH):
        print(f"HATA: Klasör bulunamadı -> {FOLDER_PATH}")
        print("Lütfen FOLDER_PATH değişkenini doğru ayarlayın.")
        return

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    image_files = [f for f in os.listdir(FOLDER_PATH) if f.lower().endswith(valid_extensions)]
    image_files.sort()

    if not image_files:
        print("Klasörde resim bulunamadı.")
        return

    print(f"Toplam {len(image_files)} resim bulundu. Analiz başlıyor...\n")

    for img_name in image_files:
        full_path = os.path.join(FOLDER_PATH, img_name)

        current_image = cv2.imread(full_path)

        if current_image is None:
            print(f"Resim okunamadı: {img_name}")
            continue

        if current_image.shape[0] > 800:
            scale = 800 / current_image.shape[0]
            dim = (int(current_image.shape[1] * scale), 800)
            current_image = cv2.resize(current_image, dim)

        display_img = current_image.copy()
        h, w = current_image.shape[:2]
        gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

        # ------------------------------------------------------------
        # 1. ADIM: KAYNAK UCUNU BUL
        # ------------------------------------------------------------
        _, mask_tip = cv2.threshold(gray, FIXED_TIP_THRESH, 255, cv2.THRESH_BINARY)
        contours_tip, _ = cv2.findContours(mask_tip, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        found_tip = False
        scan_start_y = 0
        tip_center_x = 0

        limit_outer_left = 0
        limit_outer_right = w
        final_left = 0
        final_right = w

        if contours_tip:
            largest_tip = max(contours_tip, key=cv2.contourArea)
            if cv2.contourArea(largest_tip) > 20:
                tx, ty, tw, th = cv2.boundingRect(largest_tip)
                tip_center_x = tx + tw // 2
                found_tip = True

                limit_outer_left = max(0, tip_center_x - MAX_SEARCH_WIDTH)
                limit_outer_right = min(w, tip_center_x + MAX_SEARCH_WIDTH)

                final_left = limit_outer_left
                final_right = limit_outer_right

                # Dinamik Başlangıç
                search_y_start = ty + th + OFFSET_VAL

                if search_y_start < h:
                    roi_glare_search = gray[search_y_start:h, int(limit_outer_left):int(limit_outer_right)]
                    if roi_glare_search.size > 0:
                        mask_glare = roi_glare_search > BRIGHT_THRESH
                        if np.any(mask_glare):
                            k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                            k_dilate = np.ones((k_dim, k_dim), np.uint8)
                            mask_glare_dilated = cv2.dilate(mask_glare.astype(np.uint8), k_dilate, iterations=1)
                            glare_ys, _ = np.where(mask_glare_dilated > 0)
                            if glare_ys.size > 0:
                                max_glare_y_local = np.max(glare_ys)
                                scan_start_y = search_y_start + max_glare_y_local
                            else:
                                scan_start_y = search_y_start
                        else:
                            scan_start_y = search_y_start
                    else:
                        scan_start_y = search_y_start
                else:
                    scan_start_y = h

                scan_start_y = min(scan_start_y, h - 1)
                scan_start_y = int(scan_start_y)
                cv2.rectangle(display_img, (tx, ty), (tx+tw, ty+th), (0, 0, 255), 2)

        # ------------------------------------------------------------
        # 2. ADIM: ANALİZ VE SINIR BELİRLEME
        # ------------------------------------------------------------
        debug_mask = np.zeros_like(gray)
        defect_count = 0

        if found_tip and scan_start_y < h:
            roi_gray = gray[scan_start_y:h, int(limit_outer_left):int(limit_outer_right)]

            if roi_gray.size > 0:
                h_roi, w_roi = roi_gray.shape

                # Görüntü İşleme
                clahe = cv2.createCLAHE(clipLimit=COMMON_CONTRAST, tileGridSize=(8,8))
                roi_enhanced = clahe.apply(roi_gray)
                roi_blurred = cv2.medianBlur(roi_enhanced, 5)
                roi_binary = cv2.adaptiveThreshold(
                    roi_blurred, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV, 21, COMMON_SENSITIVITY
                )

                if MORPH_SIZE > 0:
                    k_size = 2 * MORPH_SIZE + 1
                    kernel_morph = np.ones((k_size, k_size), np.uint8)
                    roi_binary = cv2.morphologyEx(roi_binary, cv2.MORPH_OPEN, kernel_morph)

                # Parlaklık Maskesi
                mask_bright_raw = roi_gray > BRIGHT_THRESH
                mask_exclude_bright = np.zeros_like(roi_gray, dtype=np.uint8)
                if np.any(mask_bright_raw):
                    k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                    kernel_dilate = np.ones((k_dim, k_dim), np.uint8)
                    mask_bright_uint8 = (mask_bright_raw * 255).astype(np.uint8)
                    mask_exclude_bright = cv2.dilate(mask_bright_uint8, kernel_dilate, iterations=1)

                # Sınırları Belirle
                roi_center_x = w_roi // 2
                safe_zone_start = roi_center_x - CENTER_SAFE_ZONE
                safe_zone_end = roi_center_x + CENTER_SAFE_ZONE

                # Gri Referans Çizgiler
                cv2.line(display_img, (int(limit_outer_left + safe_zone_start), scan_start_y),
                         (int(limit_outer_left + safe_zone_start), h), (80,80,80), 1)
                cv2.line(display_img, (int(limit_outer_left + safe_zone_end), scan_start_y),
                         (int(limit_outer_left + safe_zone_end), h), (80,80,80), 1)

                cnts, _ = cv2.findContours(roi_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                left_wall_candidates = []
                right_wall_candidates = []
                defects_to_process = []

                for cnt in cnts:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    cnt_center_x = bx + bw // 2

                    if mask_exclude_bright[by + bh//2, cnt_center_x] > 0: continue
                    if cv2.contourArea(cnt) < MIN_DEFECT_AREA: continue

                    # Sınıflandırma
                    if (bx + bw) < safe_zone_start:
                        if bh > EDGE_MIN_HEIGHT:
                            left_wall_candidates.append(bx + bw)
                    elif bx > safe_zone_end:
                        if bh > EDGE_MIN_HEIGHT:
                            right_wall_candidates.append(bx)
                    else:
                        defects_to_process.append(cnt)

                # Final Sınırlar
                if left_wall_candidates:
                    final_left = limit_outer_left + max(left_wall_candidates)
                else:
                    final_left = limit_outer_left

                if right_wall_candidates:
                    final_right = limit_outer_left + min(right_wall_candidates)
                else:
                    final_right = limit_outer_right

                # Hataları Çiz
                for cnt in defects_to_process:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    global_bx = limit_outer_left + bx
                    global_cnt_center = global_bx + bw // 2

                    if final_left < global_cnt_center < final_right:
                        defect_count += 1

                        cnt_shifted = cnt + np.array([int(limit_outer_left), scan_start_y])

                        perimeter = cv2.arcLength(cnt, True)
                        if perimeter == 0: continue
                        area = cv2.contourArea(cnt)
                        circularity = 4 * np.pi * (area / (perimeter * perimeter))
                        aspect_ratio = float(bw) / bh if bh > 0 else 0

                        label = ""
                        color = (0,0,0)
                        if circularity > CIRCULARITY_THRESH:
                            label = "Gozenek"
                            color = (0, 0, 255)
                            (cx, cy), radius = cv2.minEnclosingCircle(cnt_shifted)
                            cv2.circle(display_img, (int(cx), int(cy)), int(radius)+2, color, 2)
                        elif aspect_ratio > CRACK_RATIO or aspect_ratio < (1/CRACK_RATIO):
                            label = "Catlak"
                            color = (0, 255, 255)
                            cv2.rectangle(display_img, (global_bx, by+scan_start_y), (global_bx+bw, by+bh+scan_start_y), color, 2)
                        else:
                            label = "Bozuk"
                            color = (255, 0, 255)
                            cv2.drawContours(display_img, [cnt_shifted], -1, color, 2)

                        cv2.putText(display_img, label, (global_bx, by+scan_start_y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # ------------------------------------------------------------
        # 3. GÖRSELLEŞTİRME
        # ------------------------------------------------------------
        if scan_start_y > 0:
            overlay_top = display_img.copy()
            cv2.rectangle(overlay_top, (0, 0), (w, scan_start_y), (40, 40, 40), -1)
            cv2.addWeighted(overlay_top, 0.3, display_img, 0.7, 0, display_img)
            cv2.line(display_img, (0, scan_start_y), (w, scan_start_y), (0, 0, 255), 2)

        cv2.line(display_img, (int(final_left), scan_start_y), (int(final_left), h), (0, 165, 255), 3)
        cv2.line(display_img, (int(final_right), scan_start_y), (int(final_right), h), (0, 165, 255), 3)

        debug_view = cv2.cvtColor(debug_mask, cv2.COLOR_GRAY2BGR)

        # Bilgi Yazısı
        cv2.rectangle(display_img, (0, 0), (w, 40), (0,0,0), -1)
        info_text = f"Dosya: {img_name} | Hata Sayisi: {defect_count}"
        cv2.putText(display_img, info_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        combined = np.hstack((debug_view, display_img))

        # Ekrana Bas (Colab)
        print(f"\n--- {img_name} Analizi ---")
        cv2_imshow(combined)

if __name__ == "__main__":
    process_images_in_colab()

Output hidden; open in https://colab.research.google.com to view.

In [2]:

import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import drive

# ==========================================
# 1. GOOGLE DRIVE BAĞLANTISI
# ==========================================
# Eğer drive zaten bağlı değilse bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- DİKKAT: Buranın images/ kısmının sonuna hata türünü yazınız: (saglam,bozuk,delik,kalinti,yarik)
FOLDER_PATH = "/content/drive/MyDrive/images/yarik"
# ==========================================

# ==========================================
# 2. SABİT PARAMETRELER (AYARLAR)
# ==========================================
# --- Genel Ayarlar ---
MAX_SEARCH_WIDTH = 127      # Kaynak dikişinin aranacağı maksimum genişlik
FIXED_TIP_THRESH = 200      # Kaynak ucunu bulmak için parlaklık eşiği
OFFSET_VAL = 10             # Kaynak ucundan sonra başlama boşluğu

# --- Görüntü İşleme / Hata Bulma ---
COMMON_SENSITIVITY = 5      # Adaptive Threshold (Düşük = Daha hassas)
COMMON_CONTRAST = 2.0       # CLAHE Clip Limit
MORPH_SIZE = 1              # Gürültü temizleme boyutu
MIN_DEFECT_AREA = 127       # Min hata alanı

# --- Kenar Duvarı Mantığı ---
CENTER_SAFE_ZONE = 40       # Merkeze ±40px mesafedeki her şey HATA kabul edilir.
EDGE_MIN_HEIGHT = 60        # Dışarıdaki şeklin "Duvar" sayılması için min boy.

# --- Parlaklık / Glare ---
BRIGHT_THRESH = 130         # Parlak yerler maskelenir
IGNORE_RADIUS_BRIGHT = 10   # Parlak alan güvenlik çemberi

# --- Sınıflandırma ---
CIRCULARITY_THRESH = 0.67   # Yuvarlaklık eşiği
CRACK_RATIO = 3.9           # En/Boy oranı

# ==========================================
# 3. İŞLEM FONKSİYONU
# ==========================================
def process_images_in_colab():
    if not os.path.exists(FOLDER_PATH):
        print(f"HATA: Klasör bulunamadı -> {FOLDER_PATH}")
        print("Lütfen FOLDER_PATH değişkenini doğru ayarlayın.")
        return

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    image_files = [f for f in os.listdir(FOLDER_PATH) if f.lower().endswith(valid_extensions)]
    image_files.sort()

    if not image_files:
        print("Klasörde resim bulunamadı.")
        return

    print(f"Toplam {len(image_files)} resim bulundu. Analiz başlıyor...\n")

    for img_name in image_files:
        full_path = os.path.join(FOLDER_PATH, img_name)

        current_image = cv2.imread(full_path)

        if current_image is None:
            print(f"Resim okunamadı: {img_name}")
            continue

        if current_image.shape[0] > 800:
            scale = 800 / current_image.shape[0]
            dim = (int(current_image.shape[1] * scale), 800)
            current_image = cv2.resize(current_image, dim)

        display_img = current_image.copy()
        h, w = current_image.shape[:2]
        gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

        # ------------------------------------------------------------
        # 1. ADIM: KAYNAK UCUNU BUL
        # ------------------------------------------------------------
        _, mask_tip = cv2.threshold(gray, FIXED_TIP_THRESH, 255, cv2.THRESH_BINARY)
        contours_tip, _ = cv2.findContours(mask_tip, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        found_tip = False
        scan_start_y = 0
        tip_center_x = 0

        limit_outer_left = 0
        limit_outer_right = w
        final_left = 0
        final_right = w

        if contours_tip:
            largest_tip = max(contours_tip, key=cv2.contourArea)
            if cv2.contourArea(largest_tip) > 20:
                tx, ty, tw, th = cv2.boundingRect(largest_tip)
                tip_center_x = tx + tw // 2
                found_tip = True

                limit_outer_left = max(0, tip_center_x - MAX_SEARCH_WIDTH)
                limit_outer_right = min(w, tip_center_x + MAX_SEARCH_WIDTH)

                final_left = limit_outer_left
                final_right = limit_outer_right

                # Dinamik Başlangıç
                search_y_start = ty + th + OFFSET_VAL

                if search_y_start < h:
                    roi_glare_search = gray[search_y_start:h, int(limit_outer_left):int(limit_outer_right)]
                    if roi_glare_search.size > 0:
                        mask_glare = roi_glare_search > BRIGHT_THRESH
                        if np.any(mask_glare):
                            k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                            k_dilate = np.ones((k_dim, k_dim), np.uint8)
                            mask_glare_dilated = cv2.dilate(mask_glare.astype(np.uint8), k_dilate, iterations=1)
                            glare_ys, _ = np.where(mask_glare_dilated > 0)
                            if glare_ys.size > 0:
                                max_glare_y_local = np.max(glare_ys)
                                scan_start_y = search_y_start + max_glare_y_local
                            else:
                                scan_start_y = search_y_start
                        else:
                            scan_start_y = search_y_start
                    else:
                        scan_start_y = search_y_start
                else:
                    scan_start_y = h

                scan_start_y = min(scan_start_y, h - 1)
                scan_start_y = int(scan_start_y)
                cv2.rectangle(display_img, (tx, ty), (tx+tw, ty+th), (0, 0, 255), 2)

        # ------------------------------------------------------------
        # 2. ADIM: ANALİZ VE SINIR BELİRLEME
        # ------------------------------------------------------------
        debug_mask = np.zeros_like(gray)
        defect_count = 0

        if found_tip and scan_start_y < h:
            roi_gray = gray[scan_start_y:h, int(limit_outer_left):int(limit_outer_right)]

            if roi_gray.size > 0:
                h_roi, w_roi = roi_gray.shape

                # Görüntü İşleme
                clahe = cv2.createCLAHE(clipLimit=COMMON_CONTRAST, tileGridSize=(8,8))
                roi_enhanced = clahe.apply(roi_gray)
                roi_blurred = cv2.medianBlur(roi_enhanced, 5)
                roi_binary = cv2.adaptiveThreshold(
                    roi_blurred, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV, 21, COMMON_SENSITIVITY
                )

                if MORPH_SIZE > 0:
                    k_size = 2 * MORPH_SIZE + 1
                    kernel_morph = np.ones((k_size, k_size), np.uint8)
                    roi_binary = cv2.morphologyEx(roi_binary, cv2.MORPH_OPEN, kernel_morph)

                # Parlaklık Maskesi
                mask_bright_raw = roi_gray > BRIGHT_THRESH
                mask_exclude_bright = np.zeros_like(roi_gray, dtype=np.uint8)
                if np.any(mask_bright_raw):
                    k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                    kernel_dilate = np.ones((k_dim, k_dim), np.uint8)
                    mask_bright_uint8 = (mask_bright_raw * 255).astype(np.uint8)
                    mask_exclude_bright = cv2.dilate(mask_bright_uint8, kernel_dilate, iterations=1)

                # Sınırları Belirle
                roi_center_x = w_roi // 2
                safe_zone_start = roi_center_x - CENTER_SAFE_ZONE
                safe_zone_end = roi_center_x + CENTER_SAFE_ZONE

                # Gri Referans Çizgiler
                cv2.line(display_img, (int(limit_outer_left + safe_zone_start), scan_start_y),
                         (int(limit_outer_left + safe_zone_start), h), (80,80,80), 1)
                cv2.line(display_img, (int(limit_outer_left + safe_zone_end), scan_start_y),
                         (int(limit_outer_left + safe_zone_end), h), (80,80,80), 1)

                cnts, _ = cv2.findContours(roi_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                left_wall_candidates = []
                right_wall_candidates = []
                defects_to_process = []

                for cnt in cnts:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    cnt_center_x = bx + bw // 2

                    if mask_exclude_bright[by + bh//2, cnt_center_x] > 0: continue
                    if cv2.contourArea(cnt) < MIN_DEFECT_AREA: continue

                    # Sınıflandırma
                    if (bx + bw) < safe_zone_start:
                        if bh > EDGE_MIN_HEIGHT:
                            left_wall_candidates.append(bx + bw)
                    elif bx > safe_zone_end:
                        if bh > EDGE_MIN_HEIGHT:
                            right_wall_candidates.append(bx)
                    else:
                        defects_to_process.append(cnt)

                # Final Sınırlar
                if left_wall_candidates:
                    final_left = limit_outer_left + max(left_wall_candidates)
                else:
                    final_left = limit_outer_left

                if right_wall_candidates:
                    final_right = limit_outer_left + min(right_wall_candidates)
                else:
                    final_right = limit_outer_right

                # Hataları Çiz
                for cnt in defects_to_process:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    global_bx = limit_outer_left + bx
                    global_cnt_center = global_bx + bw // 2

                    if final_left < global_cnt_center < final_right:
                        defect_count += 1

                        cnt_shifted = cnt + np.array([int(limit_outer_left), scan_start_y])

                        perimeter = cv2.arcLength(cnt, True)
                        if perimeter == 0: continue
                        area = cv2.contourArea(cnt)
                        circularity = 4 * np.pi * (area / (perimeter * perimeter))
                        aspect_ratio = float(bw) / bh if bh > 0 else 0

                        label = ""
                        color = (0,0,0)
                        if circularity > CIRCULARITY_THRESH:
                            label = "Gozenek"
                            color = (0, 0, 255)
                            (cx, cy), radius = cv2.minEnclosingCircle(cnt_shifted)
                            cv2.circle(display_img, (int(cx), int(cy)), int(radius)+2, color, 2)
                        elif aspect_ratio > CRACK_RATIO or aspect_ratio < (1/CRACK_RATIO):
                            label = "Catlak"
                            color = (0, 255, 255)
                            cv2.rectangle(display_img, (global_bx, by+scan_start_y), (global_bx+bw, by+bh+scan_start_y), color, 2)
                        else:
                            label = "Bozuk"
                            color = (255, 0, 255)
                            cv2.drawContours(display_img, [cnt_shifted], -1, color, 2)

                        cv2.putText(display_img, label, (global_bx, by+scan_start_y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # ------------------------------------------------------------
        # 3. GÖRSELLEŞTİRME
        # ------------------------------------------------------------
        if scan_start_y > 0:
            overlay_top = display_img.copy()
            cv2.rectangle(overlay_top, (0, 0), (w, scan_start_y), (40, 40, 40), -1)
            cv2.addWeighted(overlay_top, 0.3, display_img, 0.7, 0, display_img)
            cv2.line(display_img, (0, scan_start_y), (w, scan_start_y), (0, 0, 255), 2)

        cv2.line(display_img, (int(final_left), scan_start_y), (int(final_left), h), (0, 165, 255), 3)
        cv2.line(display_img, (int(final_right), scan_start_y), (int(final_right), h), (0, 165, 255), 3)

        debug_view = cv2.cvtColor(debug_mask, cv2.COLOR_GRAY2BGR)

        # Bilgi Yazısı
        cv2.rectangle(display_img, (0, 0), (w, 40), (0,0,0), -1)
        info_text = f"Dosya: {img_name} | Hata Sayisi: {defect_count}"
        cv2.putText(display_img, info_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        combined = np.hstack((debug_view, display_img))

        # Ekrana Bas (Colab)
        print(f"\n--- {img_name} Analizi ---")
        cv2_imshow(combined)

if __name__ == "__main__":
    process_images_in_colab()

Output hidden; open in https://colab.research.google.com to view.

In [3]:

import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import drive

# ==========================================
# 1. GOOGLE DRIVE BAĞLANTISI
# ==========================================
# Eğer drive zaten bağlı değilse bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- DİKKAT: Buranın images/ kısmının sonuna hata türünü yazınız: (saglam,bozuk,delik,kalinti,yarik)
FOLDER_PATH = "/content/drive/MyDrive/images/kalinti"
# ==========================================

# ==========================================
# 2. SABİT PARAMETRELER (AYARLAR)
# ==========================================
# --- Genel Ayarlar ---
MAX_SEARCH_WIDTH = 127      # Kaynak dikişinin aranacağı maksimum genişlik
FIXED_TIP_THRESH = 200      # Kaynak ucunu bulmak için parlaklık eşiği
OFFSET_VAL = 10             # Kaynak ucundan sonra başlama boşluğu

# --- Görüntü İşleme / Hata Bulma ---
COMMON_SENSITIVITY = 5      # Adaptive Threshold (Düşük = Daha hassas)
COMMON_CONTRAST = 2.0       # CLAHE Clip Limit
MORPH_SIZE = 1              # Gürültü temizleme boyutu
MIN_DEFECT_AREA = 127       # Min hata alanı

# --- Kenar Duvarı Mantığı ---
CENTER_SAFE_ZONE = 40       # Merkeze ±40px mesafedeki her şey HATA kabul edilir.
EDGE_MIN_HEIGHT = 60        # Dışarıdaki şeklin "Duvar" sayılması için min boy.

# --- Parlaklık / Glare ---
BRIGHT_THRESH = 130         # Parlak yerler maskelenir
IGNORE_RADIUS_BRIGHT = 10   # Parlak alan güvenlik çemberi

# --- Sınıflandırma ---
CIRCULARITY_THRESH = 0.67   # Yuvarlaklık eşiği
CRACK_RATIO = 3.9           # En/Boy oranı

# ==========================================
# 3. İŞLEM FONKSİYONU
# ==========================================
def process_images_in_colab():
    if not os.path.exists(FOLDER_PATH):
        print(f"HATA: Klasör bulunamadı -> {FOLDER_PATH}")
        print("Lütfen FOLDER_PATH değişkenini doğru ayarlayın.")
        return

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    image_files = [f for f in os.listdir(FOLDER_PATH) if f.lower().endswith(valid_extensions)]
    image_files.sort()

    if not image_files:
        print("Klasörde resim bulunamadı.")
        return

    print(f"Toplam {len(image_files)} resim bulundu. Analiz başlıyor...\n")

    for img_name in image_files:
        full_path = os.path.join(FOLDER_PATH, img_name)

        current_image = cv2.imread(full_path)

        if current_image is None:
            print(f"Resim okunamadı: {img_name}")
            continue

        if current_image.shape[0] > 800:
            scale = 800 / current_image.shape[0]
            dim = (int(current_image.shape[1] * scale), 800)
            current_image = cv2.resize(current_image, dim)

        display_img = current_image.copy()
        h, w = current_image.shape[:2]
        gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

        # ------------------------------------------------------------
        # 1. ADIM: KAYNAK UCUNU BUL
        # ------------------------------------------------------------
        _, mask_tip = cv2.threshold(gray, FIXED_TIP_THRESH, 255, cv2.THRESH_BINARY)
        contours_tip, _ = cv2.findContours(mask_tip, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        found_tip = False
        scan_start_y = 0
        tip_center_x = 0

        limit_outer_left = 0
        limit_outer_right = w
        final_left = 0
        final_right = w

        if contours_tip:
            largest_tip = max(contours_tip, key=cv2.contourArea)
            if cv2.contourArea(largest_tip) > 20:
                tx, ty, tw, th = cv2.boundingRect(largest_tip)
                tip_center_x = tx + tw // 2
                found_tip = True

                limit_outer_left = max(0, tip_center_x - MAX_SEARCH_WIDTH)
                limit_outer_right = min(w, tip_center_x + MAX_SEARCH_WIDTH)

                final_left = limit_outer_left
                final_right = limit_outer_right

                # Dinamik Başlangıç
                search_y_start = ty + th + OFFSET_VAL

                if search_y_start < h:
                    roi_glare_search = gray[search_y_start:h, int(limit_outer_left):int(limit_outer_right)]
                    if roi_glare_search.size > 0:
                        mask_glare = roi_glare_search > BRIGHT_THRESH
                        if np.any(mask_glare):
                            k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                            k_dilate = np.ones((k_dim, k_dim), np.uint8)
                            mask_glare_dilated = cv2.dilate(mask_glare.astype(np.uint8), k_dilate, iterations=1)
                            glare_ys, _ = np.where(mask_glare_dilated > 0)
                            if glare_ys.size > 0:
                                max_glare_y_local = np.max(glare_ys)
                                scan_start_y = search_y_start + max_glare_y_local
                            else:
                                scan_start_y = search_y_start
                        else:
                            scan_start_y = search_y_start
                    else:
                        scan_start_y = search_y_start
                else:
                    scan_start_y = h

                scan_start_y = min(scan_start_y, h - 1)
                scan_start_y = int(scan_start_y)
                cv2.rectangle(display_img, (tx, ty), (tx+tw, ty+th), (0, 0, 255), 2)

        # ------------------------------------------------------------
        # 2. ADIM: ANALİZ VE SINIR BELİRLEME
        # ------------------------------------------------------------
        debug_mask = np.zeros_like(gray)
        defect_count = 0

        if found_tip and scan_start_y < h:
            roi_gray = gray[scan_start_y:h, int(limit_outer_left):int(limit_outer_right)]

            if roi_gray.size > 0:
                h_roi, w_roi = roi_gray.shape

                # Görüntü İşleme
                clahe = cv2.createCLAHE(clipLimit=COMMON_CONTRAST, tileGridSize=(8,8))
                roi_enhanced = clahe.apply(roi_gray)
                roi_blurred = cv2.medianBlur(roi_enhanced, 5)
                roi_binary = cv2.adaptiveThreshold(
                    roi_blurred, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV, 21, COMMON_SENSITIVITY
                )

                if MORPH_SIZE > 0:
                    k_size = 2 * MORPH_SIZE + 1
                    kernel_morph = np.ones((k_size, k_size), np.uint8)
                    roi_binary = cv2.morphologyEx(roi_binary, cv2.MORPH_OPEN, kernel_morph)

                # Parlaklık Maskesi
                mask_bright_raw = roi_gray > BRIGHT_THRESH
                mask_exclude_bright = np.zeros_like(roi_gray, dtype=np.uint8)
                if np.any(mask_bright_raw):
                    k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                    kernel_dilate = np.ones((k_dim, k_dim), np.uint8)
                    mask_bright_uint8 = (mask_bright_raw * 255).astype(np.uint8)
                    mask_exclude_bright = cv2.dilate(mask_bright_uint8, kernel_dilate, iterations=1)

                # Sınırları Belirle
                roi_center_x = w_roi // 2
                safe_zone_start = roi_center_x - CENTER_SAFE_ZONE
                safe_zone_end = roi_center_x + CENTER_SAFE_ZONE

                # Gri Referans Çizgiler
                cv2.line(display_img, (int(limit_outer_left + safe_zone_start), scan_start_y),
                         (int(limit_outer_left + safe_zone_start), h), (80,80,80), 1)
                cv2.line(display_img, (int(limit_outer_left + safe_zone_end), scan_start_y),
                         (int(limit_outer_left + safe_zone_end), h), (80,80,80), 1)

                cnts, _ = cv2.findContours(roi_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                left_wall_candidates = []
                right_wall_candidates = []
                defects_to_process = []

                for cnt in cnts:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    cnt_center_x = bx + bw // 2

                    if mask_exclude_bright[by + bh//2, cnt_center_x] > 0: continue
                    if cv2.contourArea(cnt) < MIN_DEFECT_AREA: continue

                    # Sınıflandırma
                    if (bx + bw) < safe_zone_start:
                        if bh > EDGE_MIN_HEIGHT:
                            left_wall_candidates.append(bx + bw)
                    elif bx > safe_zone_end:
                        if bh > EDGE_MIN_HEIGHT:
                            right_wall_candidates.append(bx)
                    else:
                        defects_to_process.append(cnt)

                # Final Sınırlar
                if left_wall_candidates:
                    final_left = limit_outer_left + max(left_wall_candidates)
                else:
                    final_left = limit_outer_left

                if right_wall_candidates:
                    final_right = limit_outer_left + min(right_wall_candidates)
                else:
                    final_right = limit_outer_right

                # Hataları Çiz
                for cnt in defects_to_process:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    global_bx = limit_outer_left + bx
                    global_cnt_center = global_bx + bw // 2

                    if final_left < global_cnt_center < final_right:
                        defect_count += 1

                        cnt_shifted = cnt + np.array([int(limit_outer_left), scan_start_y])

                        perimeter = cv2.arcLength(cnt, True)
                        if perimeter == 0: continue
                        area = cv2.contourArea(cnt)
                        circularity = 4 * np.pi * (area / (perimeter * perimeter))
                        aspect_ratio = float(bw) / bh if bh > 0 else 0

                        label = ""
                        color = (0,0,0)
                        if circularity > CIRCULARITY_THRESH:
                            label = "Gozenek"
                            color = (0, 0, 255)
                            (cx, cy), radius = cv2.minEnclosingCircle(cnt_shifted)
                            cv2.circle(display_img, (int(cx), int(cy)), int(radius)+2, color, 2)
                        elif aspect_ratio > CRACK_RATIO or aspect_ratio < (1/CRACK_RATIO):
                            label = "Catlak"
                            color = (0, 255, 255)
                            cv2.rectangle(display_img, (global_bx, by+scan_start_y), (global_bx+bw, by+bh+scan_start_y), color, 2)
                        else:
                            label = "Bozuk"
                            color = (255, 0, 255)
                            cv2.drawContours(display_img, [cnt_shifted], -1, color, 2)

                        cv2.putText(display_img, label, (global_bx, by+scan_start_y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # ------------------------------------------------------------
        # 3. GÖRSELLEŞTİRME
        # ------------------------------------------------------------
        if scan_start_y > 0:
            overlay_top = display_img.copy()
            cv2.rectangle(overlay_top, (0, 0), (w, scan_start_y), (40, 40, 40), -1)
            cv2.addWeighted(overlay_top, 0.3, display_img, 0.7, 0, display_img)
            cv2.line(display_img, (0, scan_start_y), (w, scan_start_y), (0, 0, 255), 2)

        cv2.line(display_img, (int(final_left), scan_start_y), (int(final_left), h), (0, 165, 255), 3)
        cv2.line(display_img, (int(final_right), scan_start_y), (int(final_right), h), (0, 165, 255), 3)

        debug_view = cv2.cvtColor(debug_mask, cv2.COLOR_GRAY2BGR)

        # Bilgi Yazısı
        cv2.rectangle(display_img, (0, 0), (w, 40), (0,0,0), -1)
        info_text = f"Dosya: {img_name} | Hata Sayisi: {defect_count}"
        cv2.putText(display_img, info_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        combined = np.hstack((debug_view, display_img))

        # Ekrana Bas (Colab)
        print(f"\n--- {img_name} Analizi ---")
        cv2_imshow(combined)

if __name__ == "__main__":
    process_images_in_colab()

Output hidden; open in https://colab.research.google.com to view.

In [4]:

import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import drive

# ==========================================
# 1. GOOGLE DRIVE BAĞLANTISI
# ==========================================
# Eğer drive zaten bağlı değilse bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- DİKKAT: Buranın images/ kısmının sonuna hata türünü yazınız: (saglam,bozuk,delik,kalinti,yarik)
FOLDER_PATH = "/content/drive/MyDrive/images/bozuk"
# ==========================================

# ==========================================
# 2. SABİT PARAMETRELER (AYARLAR)
# ==========================================
# --- Genel Ayarlar ---
MAX_SEARCH_WIDTH = 127      # Kaynak dikişinin aranacağı maksimum genişlik
FIXED_TIP_THRESH = 200      # Kaynak ucunu bulmak için parlaklık eşiği
OFFSET_VAL = 10             # Kaynak ucundan sonra başlama boşluğu

# --- Görüntü İşleme / Hata Bulma ---
COMMON_SENSITIVITY = 5      # Adaptive Threshold (Düşük = Daha hassas)
COMMON_CONTRAST = 2.0       # CLAHE Clip Limit
MORPH_SIZE = 1              # Gürültü temizleme boyutu
MIN_DEFECT_AREA = 127       # Min hata alanı

# --- Kenar Duvarı Mantığı ---
CENTER_SAFE_ZONE = 40       # Merkeze ±40px mesafedeki her şey HATA kabul edilir.
EDGE_MIN_HEIGHT = 60        # Dışarıdaki şeklin "Duvar" sayılması için min boy.

# --- Parlaklık / Glare ---
BRIGHT_THRESH = 130         # Parlak yerler maskelenir
IGNORE_RADIUS_BRIGHT = 10   # Parlak alan güvenlik çemberi

# --- Sınıflandırma ---
CIRCULARITY_THRESH = 0.67   # Yuvarlaklık eşiği
CRACK_RATIO = 3.9           # En/Boy oranı

# ==========================================
# 3. İŞLEM FONKSİYONU
# ==========================================
def process_images_in_colab():
    if not os.path.exists(FOLDER_PATH):
        print(f"HATA: Klasör bulunamadı -> {FOLDER_PATH}")
        print("Lütfen FOLDER_PATH değişkenini doğru ayarlayın.")
        return

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    image_files = [f for f in os.listdir(FOLDER_PATH) if f.lower().endswith(valid_extensions)]
    image_files.sort()

    if not image_files:
        print("Klasörde resim bulunamadı.")
        return

    print(f"Toplam {len(image_files)} resim bulundu. Analiz başlıyor...\n")

    for img_name in image_files:
        full_path = os.path.join(FOLDER_PATH, img_name)

        current_image = cv2.imread(full_path)

        if current_image is None:
            print(f"Resim okunamadı: {img_name}")
            continue

        if current_image.shape[0] > 800:
            scale = 800 / current_image.shape[0]
            dim = (int(current_image.shape[1] * scale), 800)
            current_image = cv2.resize(current_image, dim)

        display_img = current_image.copy()
        h, w = current_image.shape[:2]
        gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

        # ------------------------------------------------------------
        # 1. ADIM: KAYNAK UCUNU BUL
        # ------------------------------------------------------------
        _, mask_tip = cv2.threshold(gray, FIXED_TIP_THRESH, 255, cv2.THRESH_BINARY)
        contours_tip, _ = cv2.findContours(mask_tip, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        found_tip = False
        scan_start_y = 0
        tip_center_x = 0

        limit_outer_left = 0
        limit_outer_right = w
        final_left = 0
        final_right = w

        if contours_tip:
            largest_tip = max(contours_tip, key=cv2.contourArea)
            if cv2.contourArea(largest_tip) > 20:
                tx, ty, tw, th = cv2.boundingRect(largest_tip)
                tip_center_x = tx + tw // 2
                found_tip = True

                limit_outer_left = max(0, tip_center_x - MAX_SEARCH_WIDTH)
                limit_outer_right = min(w, tip_center_x + MAX_SEARCH_WIDTH)

                final_left = limit_outer_left
                final_right = limit_outer_right

                # Dinamik Başlangıç
                search_y_start = ty + th + OFFSET_VAL

                if search_y_start < h:
                    roi_glare_search = gray[search_y_start:h, int(limit_outer_left):int(limit_outer_right)]
                    if roi_glare_search.size > 0:
                        mask_glare = roi_glare_search > BRIGHT_THRESH
                        if np.any(mask_glare):
                            k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                            k_dilate = np.ones((k_dim, k_dim), np.uint8)
                            mask_glare_dilated = cv2.dilate(mask_glare.astype(np.uint8), k_dilate, iterations=1)
                            glare_ys, _ = np.where(mask_glare_dilated > 0)
                            if glare_ys.size > 0:
                                max_glare_y_local = np.max(glare_ys)
                                scan_start_y = search_y_start + max_glare_y_local
                            else:
                                scan_start_y = search_y_start
                        else:
                            scan_start_y = search_y_start
                    else:
                        scan_start_y = search_y_start
                else:
                    scan_start_y = h

                scan_start_y = min(scan_start_y, h - 1)
                scan_start_y = int(scan_start_y)
                cv2.rectangle(display_img, (tx, ty), (tx+tw, ty+th), (0, 0, 255), 2)

        # ------------------------------------------------------------
        # 2. ADIM: ANALİZ VE SINIR BELİRLEME
        # ------------------------------------------------------------
        debug_mask = np.zeros_like(gray)
        defect_count = 0

        if found_tip and scan_start_y < h:
            roi_gray = gray[scan_start_y:h, int(limit_outer_left):int(limit_outer_right)]

            if roi_gray.size > 0:
                h_roi, w_roi = roi_gray.shape

                # Görüntü İşleme
                clahe = cv2.createCLAHE(clipLimit=COMMON_CONTRAST, tileGridSize=(8,8))
                roi_enhanced = clahe.apply(roi_gray)
                roi_blurred = cv2.medianBlur(roi_enhanced, 5)
                roi_binary = cv2.adaptiveThreshold(
                    roi_blurred, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV, 21, COMMON_SENSITIVITY
                )

                if MORPH_SIZE > 0:
                    k_size = 2 * MORPH_SIZE + 1
                    kernel_morph = np.ones((k_size, k_size), np.uint8)
                    roi_binary = cv2.morphologyEx(roi_binary, cv2.MORPH_OPEN, kernel_morph)

                # Parlaklık Maskesi
                mask_bright_raw = roi_gray > BRIGHT_THRESH
                mask_exclude_bright = np.zeros_like(roi_gray, dtype=np.uint8)
                if np.any(mask_bright_raw):
                    k_dim = 2 * IGNORE_RADIUS_BRIGHT + 1
                    kernel_dilate = np.ones((k_dim, k_dim), np.uint8)
                    mask_bright_uint8 = (mask_bright_raw * 255).astype(np.uint8)
                    mask_exclude_bright = cv2.dilate(mask_bright_uint8, kernel_dilate, iterations=1)

                # Sınırları Belirle
                roi_center_x = w_roi // 2
                safe_zone_start = roi_center_x - CENTER_SAFE_ZONE
                safe_zone_end = roi_center_x + CENTER_SAFE_ZONE

                # Gri Referans Çizgiler
                cv2.line(display_img, (int(limit_outer_left + safe_zone_start), scan_start_y),
                         (int(limit_outer_left + safe_zone_start), h), (80,80,80), 1)
                cv2.line(display_img, (int(limit_outer_left + safe_zone_end), scan_start_y),
                         (int(limit_outer_left + safe_zone_end), h), (80,80,80), 1)

                cnts, _ = cv2.findContours(roi_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                left_wall_candidates = []
                right_wall_candidates = []
                defects_to_process = []

                for cnt in cnts:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    cnt_center_x = bx + bw // 2

                    if mask_exclude_bright[by + bh//2, cnt_center_x] > 0: continue
                    if cv2.contourArea(cnt) < MIN_DEFECT_AREA: continue

                    # Sınıflandırma
                    if (bx + bw) < safe_zone_start:
                        if bh > EDGE_MIN_HEIGHT:
                            left_wall_candidates.append(bx + bw)
                    elif bx > safe_zone_end:
                        if bh > EDGE_MIN_HEIGHT:
                            right_wall_candidates.append(bx)
                    else:
                        defects_to_process.append(cnt)

                # Final Sınırlar
                if left_wall_candidates:
                    final_left = limit_outer_left + max(left_wall_candidates)
                else:
                    final_left = limit_outer_left

                if right_wall_candidates:
                    final_right = limit_outer_left + min(right_wall_candidates)
                else:
                    final_right = limit_outer_right

                # Hataları Çiz
                for cnt in defects_to_process:
                    bx, by, bw, bh = cv2.boundingRect(cnt)
                    global_bx = limit_outer_left + bx
                    global_cnt_center = global_bx + bw // 2

                    if final_left < global_cnt_center < final_right:
                        defect_count += 1

                        cnt_shifted = cnt + np.array([int(limit_outer_left), scan_start_y])

                        perimeter = cv2.arcLength(cnt, True)
                        if perimeter == 0: continue
                        area = cv2.contourArea(cnt)
                        circularity = 4 * np.pi * (area / (perimeter * perimeter))
                        aspect_ratio = float(bw) / bh if bh > 0 else 0

                        label = ""
                        color = (0,0,0)
                        if circularity > CIRCULARITY_THRESH:
                            label = "Gozenek"
                            color = (0, 0, 255)
                            (cx, cy), radius = cv2.minEnclosingCircle(cnt_shifted)
                            cv2.circle(display_img, (int(cx), int(cy)), int(radius)+2, color, 2)
                        elif aspect_ratio > CRACK_RATIO or aspect_ratio < (1/CRACK_RATIO):
                            label = "Catlak"
                            color = (0, 255, 255)
                            cv2.rectangle(display_img, (global_bx, by+scan_start_y), (global_bx+bw, by+bh+scan_start_y), color, 2)
                        else:
                            label = "Bozuk"
                            color = (255, 0, 255)
                            cv2.drawContours(display_img, [cnt_shifted], -1, color, 2)

                        cv2.putText(display_img, label, (global_bx, by+scan_start_y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # ------------------------------------------------------------
        # 3. GÖRSELLEŞTİRME
        # ------------------------------------------------------------
        if scan_start_y > 0:
            overlay_top = display_img.copy()
            cv2.rectangle(overlay_top, (0, 0), (w, scan_start_y), (40, 40, 40), -1)
            cv2.addWeighted(overlay_top, 0.3, display_img, 0.7, 0, display_img)
            cv2.line(display_img, (0, scan_start_y), (w, scan_start_y), (0, 0, 255), 2)

        cv2.line(display_img, (int(final_left), scan_start_y), (int(final_left), h), (0, 165, 255), 3)
        cv2.line(display_img, (int(final_right), scan_start_y), (int(final_right), h), (0, 165, 255), 3)

        debug_view = cv2.cvtColor(debug_mask, cv2.COLOR_GRAY2BGR)

        # Bilgi Yazısı
        cv2.rectangle(display_img, (0, 0), (w, 40), (0,0,0), -1)
        info_text = f"Dosya: {img_name} | Hata Sayisi: {defect_count}"
        cv2.putText(display_img, info_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        combined = np.hstack((debug_view, display_img))

        # Ekrana Bas (Colab)
        print(f"\n--- {img_name} Analizi ---")
        cv2_imshow(combined)

if __name__ == "__main__":
    process_images_in_colab()

Output hidden; open in https://colab.research.google.com to view.